In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import base64
from projectcrud import MongoCRUD

app = JupyterDash('Collin Zielinski Dashboard')

# Data Manipulation / Model
username = "aacuser"
password = "1234"
host = 'nv-desktop-services.apporto.com'
port = 30749
db_name = "AAC"
col_name = "animals"
shelter = MongoCRUD(username=username, password=password, host=host, port=port, db_name=db_name, collection_name=col_name)

# Fetch data from MongoDB
try:
    df = pd.DataFrame.from_records(shelter.read({}))
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)
except Exception as e:
    print(f"Failed to fetch data from MongoDB: {e}")
    df = pd.DataFrame()

encoded_image = base64.b64encode(open('Grazioso Salvare Logo.png', 'rb').read()).decode()

# Rescue type filters mapping
rescue_filters = {
    'water': {
        'breeds': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex': 'Intact Female',
        'age_weeks': (26, 156)
    },
    'mountain': {
        'breeds': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_weeks': (26, 156)
    },
    'disaster': {
        'breeds': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex': 'Intact Male',
        'age_weeks': (20, 300)
    }
}

# Layout
app.layout = html.Div([
    html.Div([
        html.A([
            html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height': '100px', 'width': 'auto'})
        ], href='http://www.snhu.edu'),
        html.P('Dashboard created by Collin Zielinski', style={'color': 'grey', 'fontSize': '12px'})
    ], style={'textAlign': 'center'}),
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',
        style={'padding': '20px'}
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns] if not df.empty else [],
        data=df.to_dict('records'),
        row_selectable='single',
        selected_rows=[],
        page_action='native',
        page_size=10,
        sort_action='native',
        style_table={'height': '300px', 'overflowY': 'auto'}
    ),
    html.Br(),
    html.Div([
        html.Div(id='map-id', className='col s12 m6'),
        html.Div(dcc.Graph(id='bar-chart-id'), className='col s12 m6')
    ]),
    html.Hr(),
])

# Callback for interactivity
@app.callback(
    [Output('datatable-id', 'data'),
     Output('map-id', "children"),
     Output('bar-chart-id', "figure")],
    [Input('rescue-type-filter', 'value'),
     Input('datatable-id', 'selected_rows')]
)
def update_visualizations(filter_type, selected_rows):
    if filter_type == 'reset' or filter_type is None or df.empty:
        filtered_data = df
    else:
        criteria = rescue_filters[filter_type]
        filtered_data = df[
            df['breed'].isin(criteria['breeds']) &
            (df['sex_upon_outcome'].str.contains(criteria['sex'], na=False)) &
            (df['age_upon_outcome_in_weeks'] >= criteria['age_weeks'][0]) &
            (df['age_upon_outcome_in_weeks'] <= criteria['age_weeks'][1])
        ]

    data_table = filtered_data.to_dict('records')

    selected_index = selected_rows[0] if selected_rows else 0
    selected_row_data = filtered_data.iloc[selected_index] if not filtered_data.empty else df.iloc[0]

    lat = selected_row_data.get('location_lat', 30.2672)
    lon = selected_row_data.get('location_long', -97.7431)
    map_component = [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(selected_row_data.get('animal_type', "No Data")),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(selected_row_data.get('name', "No Data"))
                ])
            ])
        ])
    ]

    if not filtered_data.empty:
        breeds_count = filtered_data['breed'].value_counts().reset_index()
        breeds_count.columns = ['breed', 'count']
        bar_figure = px.bar(
            breeds_count,
            x='breed',
            y='count',
            title='Distribution of Breeds in Rescue Type',
            labels={'count': 'Number of Animals'},
            height=400
        )
    else:
        bar_figure = {
            'data': [],
            'layout': {
                'title': 'No data available',
                'height': 400
            }
        }

    return data_table, map_component, bar_figure

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True)
